In [ ]:
# if need be, manually add the local project root to PYTHONPATH and move working directories

import os
import sys

project = '/' # change to local project root
sys.path.append(project)
os.chdir(project)

In [ ]:
# dependencies

import re
import datetime

import numpy as np
import pandas as pd
import pystan as stan

import nfx.lm.gibbs

In [ ]:
# helper functions

def package_coef_samples(coef_samples, node_names, covariate_names, algo_name, chain_ix, epoch_ix):

    dfs = []
    for i, (coef_samples_, node_names_) in enumerate(zip(coef_samples, node_names)):
        for j, node_names__ in enumerate(node_names_):
            df_ = pd.DataFrame(coef_samples_[:, j].T, index=covariate_names)
            df_.index = df_.index.rename('covariate')
            df_.columns = df_.columns.rename('iter')
            df_['algo'] = algo_name
            df_['chain'] = chain_ix
            df_['epoch'] = epoch_ix
            df_['level'] = i
            df_['node'] = node_names__.zfill(i + 1)
            dfs.append(df_)
    df = pd.concat(dfs).reset_index().set_index(['algo', 'chain', 'epoch', 'level', 'node', 'covariate'])
    return df

def est_post_moms(samples):

    df = pd.DataFrame(index=samples.index)
    df['mean'] = samples.mean(1)
    df['log_sd'] = np.log(samples.std(1))
    df.columns = df.columns.rename('summary')
    return df.stack().rename('value')

def package_sla_samples(samples, node_names, covariate_names, chain_ix, epoch_ix):

    coef_samples, _, _ = zip(*samples)
    coef_samples = [np.array(coef_samples_) for coef_samples_ in zip(*coef_samples)][::-1]
    return package_coef_samples(coef_samples, node_names, covariate_names, 'SLA', chain_ix, epoch_ix)

def sample_sla_table(sla_inputs, stan_model, advi_inputs, n_chains, n_epochs, covariate_names, node_names, tepoch, titer, failed_seeds):

    return pd.concat([sample_sla_sequence(sla_inputs, stan_model, advi_inputs, chain_ix, n_epochs, covariate_names, node_names, tepoch, titer) for chain_ix in range(n_chains + len(failed_seeds)) if chain_ix not in failed_seeds])

def sample_sla_sequence(sla_inputs, stan_model, advi_inputs, chain_ix, n_epochs, covariate_names, node_names, tepoch, titer):

    rng = np.random.default_rng(chain_ix)
    advi_samples = stan_model.vb(data=advi_inputs, iter=1, init=0, seed=chain_ix)
    advi_arrays = format_advi_output(advi_samples['sampler_param_names'], advi_samples['sampler_params'])
    init = ([np.mean(advi_arrays[s], 0) for s in ('coefs', 'loc_3', 'loc_2', 'loc_1', 'loc_0')], 
            [np.linalg.inv(np.mean(advi_arrays[s], 0)) for s in ('cov_4', 'cov_3', 'cov_2', 'cov_1')],
            np.mean(1 / advi_arrays['var_resid'], 0))
    n_samples = int(n_epochs * tepoch / titer) + 1
    sampler = nfx.lm.gibbs.sample_posterior(*sla_inputs, init=init, ome=rng)
    samples = [next(sampler) for _ in  range(n_samples)]
    coef_samples = package_sla_samples(samples, node_names, covariate_names, chain_ix, 0)
    return split_epochs(coef_samples, n_epochs, tepoch, titer)

def split_epochs(samples, n_epochs, tepoch, sla_titer):

    n_samples = np.int64(np.arange(1, n_epochs + 1) * (tepoch / sla_titer))
    summ_by_epoch = [est_post_moms(samples.iloc[:, (n // 2):(n)]).reset_index() for n in n_samples]
    for i, summ_ in enumerate(summ_by_epoch):
        summ_['epoch'] = i
    summ_by_epoch = pd.concat(summ_by_epoch)
    return summ_by_epoch.set_index(summ_by_epoch.columns.drop('value').tolist()).value

def package_advi_samples(samples, node_names, covariate_names, chain_ix, epoch_ix):

    coef_samples = [samples[par] for par in ('loc_0', 'loc_1', 'loc_2', 'loc_3', 'coefs')]
    coef_samples[0] = coef_samples[0][:, np.newaxis]
    return package_coef_samples(coef_samples, node_names, covariate_names, 'Stan/ADVI', chain_ix, epoch_ix)

def sample_advi_table(model, data, n_iter, n_chains, n_epochs, covariate_names, node_names):

    i = 0
    summ_coefs = []
    failed_seeds = []
    while len(summ_coefs) < n_chains:
        try:
            summ_coefs.append(sample_advi_sequence(model, data, n_iter, i, n_epochs, covariate_names, node_names))
        except RuntimeError:
            failed_seeds.append(i)
        i += 1
    return pd.concat(summ_coefs), failed_seeds

def sample_advi_sequence(model, data, n_iter, chain_ix, n_epochs, covariate_names, node_names):

    summ_coefs = [sample_advi_cell(model, data, n_iter, chain_ix, i, covariate_names, node_names) for i in range(n_epochs)]
    return pd.concat(summ_coefs)

def sample_advi_cell(model, data, n_iter, chain_ix, epoch_ix, covariate_names, node_names):

    rng = np.random.default_rng(chain_ix)
    samples = model.vb(data=data, iter=n_iter*(epoch_ix+1), init=0, seed=chain_ix, tol_rel_obj=1e-9)
    arrays = format_advi_output(samples['sampler_param_names'], samples['sampler_params'])
    coef_samples = package_advi_samples(arrays, node_names, covariate_names, chain_ix, epoch_ix)
    coef_summ = est_post_moms(coef_samples)
    return coef_summ

def format_advi_output(block_names, block_samples):

    uq_names = list(set([s.split('[')[0] for s in block_names]))
    indices = {un: np.int32([re.findall('\[(.+)\]', n)[0].split(',') if '[' in n else np.array([]) for n in block_names if un == n.split('[')[0]]) - 1 for un in uq_names}
    samples = {un: [s for n, s in zip(block_names, block_samples) if un == n.split('[')[0]] for un in uq_names}
    arrays = {un: np.empty(np.append(indices[un].max(0) + 1, 1000)) for un in uq_names}
    for un in uq_names:
        for ix, s in zip(indices[un], samples[un]):
            arrays[un][tuple(ix)] = s
        arrays[un] = np.transpose(arrays[un], [-1, *range(len(arrays[un].shape) - 1)])
    return arrays

In [ ]:
# config

covariate_names = ['housing']
n_chains = 1
n_epochs = 10
n_iter = 1024

In [ ]:
# load data

macro = pd.read_csv('paper/data/sareb_covariates.csv').set_index('time')
prices = pd.read_csv('paper/data/sareb_prices_synthetic.csv').set_index('zip')

In [ ]:
# format response

response = prices.diff(axis=1).dropna(axis=1)

In [ ]:
# format covariates

covariates = macro.loc[:, covariate_names]
covariates['_constant'] = 1
covariates['_trend'] = np.arange(covariates.shape[0])
covariates = covariates.loc[:, ['_trend', 'housing']].diff().dropna().loc[response.columns]

In [ ]:
# construct tree

indices = response.index.to_frame()
indices['lvl_1'] = indices.zip.str.slice(0, 2)
indices['lvl_2'] = indices.zip.str.slice(0, 3)
indices['lvl_3'] = indices.zip.str.slice(0, 4)
indices['lvl_4'] = indices.zip.str.slice(0, 5)
indices = indices.drop('zip', 1)
codes = indices.apply(lambda x: x.astype('category').cat.codes).astype('int64')
n_nodes = codes.max(0) + 1
parent_node_3 = codes[['lvl_4', 'lvl_3']].drop_duplicates().lvl_3
parent_node_2 = codes[['lvl_3', 'lvl_2']].drop_duplicates().lvl_2
parent_node_1 = codes[['lvl_2', 'lvl_1']].drop_duplicates().lvl_1
tree = [parent_node_3, parent_node_2, parent_node_1]
node_names = [['0'], indices.lvl_1.unique(), indices.lvl_2.unique(), indices.lvl_3.unique(), indices.lvl_4.unique()]

In [ ]:
# assemble advi inputs

advi_inputs = {
    'n_periods': response.shape[1],
    'n_leaves': response.shape[0],
    'n_covariates': covariates.shape[1],
    'n_nodes_3': parent_node_3.max() + 1,
    'n_nodes_2': parent_node_2.max() + 1,
    'n_nodes_1': parent_node_1.max() + 1,
    'parent_node_3': parent_node_3.values + 1,
    'parent_node_2': parent_node_2.values + 1,
    'parent_node_1': parent_node_1.values + 1,
    'covariates': covariates.values.T,
    'response': response.values,
    'prior_df': covariates.shape[1],
    'prior_shape': 1/2,
    'prior_rate': 1/2
}

In [ ]:
# measure advi iteration time

stan_model = stan.StanModel('paper/stan/nfx_panel_normal.stan')
t0 = datetime.datetime.now()
stan_model.vb(data=advi_inputs, seed=0, iter=n_iter * n_epochs, tol_rel_obj=1e-9)
t1 = datetime.datetime.now()
tepoch = (t1 - t0).total_seconds() / n_epochs

In [ ]:
# sample advi summaries

advi_summ, failed_seeds = sample_advi_table(stan_model, advi_inputs, n_iter, n_chains, n_epochs, ['_trend'] + covariate_names, node_names)

In [ ]:
# assemble sla inputs

sla_inputs = (response.values, covariates.values, tree)

In [ ]:
# measure sla iteration time

sampler = nfx.lm.gibbs.sample_posterior(*sla_inputs)
next(sampler)
t0 = datetime.datetime.now()
all(next(sampler) for _ in  range(100))
t1 = datetime.datetime.now()
sla_titer = (t1 - t0).total_seconds() / 100

In [ ]:
# sample sla summaries

sla_summ = sample_sla_table(sla_inputs, stan_model, advi_inputs, n_chains, n_epochs, ['_trend'] + covariate_names, node_names, tepoch, sla_titer, failed_seeds)

In [ ]:
# marginal posterior summaries for advi

advi_summ

In [ ]:
# marginal posterior summaries for sla

sla_summ